In [9]:
import os
from dotenv import load_dotenv
load_dotenv( '.env')

api_key = os.getenv("HF_TOKEN")
if not api_key:
    raise ValueError("HF_TOKEN environment variable is required")

openrouter_api_key = os.getenv("OPENROUTER_API_KEY")
if not openrouter_api_key:
    raise ValueError("OPENROUTER_API_KEY environment variable is required")


openrouter_api_base = os.getenv("OPENROUTER_API_BASE")
if not openrouter_api_base:
    raise ValueError("OPENROUTER_API_BASE environment variable is required")


anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")
if not anthropic_api_key:
    raise ValueError("ANTHROPIC_API_KEY environment variable is required")


In [2]:
from smolagents import tool 
# import packages that are used in our tools
import requests
from bs4 import BeautifulSoup
import json

@tool
def get_hugging_face_top_daily_paper() -> str:
    """
    This is a tool that returns the most upvoted paper on Hugging Face daily papers.
    It returns the title of the paper
    """
    try:
      url = "<https://huggingface.co/papers>"
      response = requests.get(url)
      response.raise_for_status()  # Raise an exception for bad status codes (4xx or 5xx)
      soup = BeautifulSoup(response.content, "html.parser")

      # Extract the title element from the JSON-like data in the "data-props" attribute
      containers = soup.find_all('div', class_='SVELTE_HYDRATER contents')
      top_paper = ""

      for container in containers:
          data_props = container.get('data-props', '')
          if data_props:
              try:
                  # Parse the JSON-like string
                  json_data = json.loads(data_props.replace('&quot;', '"'))
                  if 'dailyPapers' in json_data:
                      top_paper = json_data['dailyPapers'][0]['title']
              except json.JSONDecodeError:
                  continue

      return top_paper
    except requests.exceptions.RequestException as e:
      print(f"Error occurred while fetching the HTML: {e}")
      return None

/home/zambra/git/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from huggingface_hub import HfApi

@tool
def get_paper_id_by_title(title: str) -> str:
    """
    This is a tool that returns the arxiv paper id by its title.
    It returns the title of the paper

    Args:
        title: The paper title for which to get the id.
    """
    api = HfApi()
    papers = api.list_papers(query=title)
    if papers:
        paper = next(iter(papers))
        return paper.id
    else:
        return None

In [4]:
import arxiv

@tool
def download_paper_by_id(paper_id: str) -> None:
    """
    This tool gets the id of a paper and downloads it from arxiv. It saves the paper locally 
    in the current directory as "paper.pdf".

    Args:
        paper_id: The id of the paper to download.
    """
    paper = next(arxiv.Client().results(arxiv.Search(id_list=[paper_id])))
    paper.download_pdf(filename="paper.pdf")
    return None

In [5]:
from pypdf import PdfReader

@tool
def read_pdf_file(file_path: str) -> str:
    """
    This function reads the first three pages of a PDF file and returns its content as a string.
    Args:
        file_path: The path to the PDF file.
    Returns:
        A string containing the content of the PDF file.
    """
    content = ""
    reader = PdfReader('paper.pdf')
    print(len(reader.pages))
    pages = reader.pages[:3]
    for page in pages:
        content += page.extract_text()
    return content

In [ ]:

from smolagents import LiteLLMModel

# Configurar o modelo
model = LiteLLMModel(model_id="claude-3-5-sonnet-20240620",
                     api_key=anthropic_api_key)


messages = [
  {"role": "user", "content": [{"type": "text", "text": "Explique o conceito de gravidade"}]}
]

# Ou com mais parâmetros
response = model(
    messages,
    temperature=0.7,
    max_tokens=500
)
print(response.content)

In [ ]:
from smolagents import CodeAgent, HfApiModel

model_id = "Qwen/Qwen2.5-Coder-32B-Instruct"

#model = HfApiModel(model_id=model_id, token=api_key)
model = LiteLLMModel(model_id="claude-3-5-sonnet-20240620",
                     api_key=anthropic_api_key)

agent = CodeAgent(tools=[get_hugging_face_top_daily_paper,
                         get_paper_id_by_title,
                         download_paper_by_id,
                         read_pdf_file],
                  model=model,
                  add_base_tools=True)

agent.run(
    "Summarize today's top paper on Hugging Face daily papers by reading it.",
)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Summarize today's top paper on Hugging Face daily papers by reading it.                                         │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  top_paper_title = get_hugging_face_top_daily_paper()                                                             
  print("Top paper title:", top_paper_title)                                                                       
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Error occurred while fetching the HTML: No connection adapters were found for '<https://huggingface.co/papers>'


Execution logs:
Top paper title: None

Out: None

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 6 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("The paper 'The Stochastic Parrot on LLM’s Shoulder: A Summative Assessment of Physical Concept     
  Understanding' investigates whether LLMs truly understand their outputs. By introducing the PHYSI CO task, the   
  authors assess different levels of understanding using Bloom’s taxonomy. They find that while state-of-the-art   
  LLMs perform well on low-level tasks, they struggle with high-level tasks, demonstrating the Stochastic Parrot   
  phenomenon.")                                                                                                    
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: The paper 'The Stochastic Parrot on LLM’s Shoulder: A Summative Assessment of Physical Concept 
Understanding' investigates whether LLMs truly understand their outputs. By introducing the PHYSI CO task, the 
authors assess different levels of understanding using Bloom’s taxonomy. They find that while state-of-the-art LLMs
perform well on low-level tasks, they struggle with high-level tasks, demonstrating the Stochastic Parrot 
phenomenon.

[Step 5: Duration 17.41 seconds| Input tokens: 39,602 | Output tokens: 840]

"The paper 'The Stochastic Parrot on LLM’s Shoulder: A Summative Assessment of Physical Concept Understanding' investigates whether LLMs truly understand their outputs. By introducing the PHYSI CO task, the authors assess different levels of understanding using Bloom’s taxonomy. They find that while state-of-the-art LLMs perform well on low-level tasks, they struggle with high-level tasks, demonstrating the Stochastic Parrot phenomenon."